# Dyn and Thermo effects

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as m
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import netCDF4
# import pandas as pd
import dask
# from mpl_toolkits.basemap import Basemap
# from datetime import datetime
# from datetime import timedelta
# from scipy import stats
# import scipy.interpolate as interp 
%matplotlib inline

In [2]:
from functions import ccplot

In [3]:
from tqdm import tnrange, tqdm_notebook
from scipy import integrate
from time import sleep

In [4]:
# import seaborn as sns
# Use seaborn style defaults and set default figure size
# plt.style.use('seaborn-pastel')
# sns.set_theme(style="ticks")
# plt.style.use(['science', 'notebook'])
plt.style.use('tableau-colorblind10')

In [5]:
# Colormap selection
xr.set_options(cmap_divergent='BrBG', cmap_sequential='YlGnBu')

In [6]:
fsize = 15
tsize = 18

tdir = 'in'

major = 5.0
minor = 3.0

# plt.style.use(style)
plt.rcParams['text.usetex'] = False
plt.rcParams['font.size'] = fsize
plt.rcParams['legend.fontsize'] = tsize
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = major
plt.rcParams['ytick.minor.size'] = minor
plt.rcParams['axes.linewidth'] = 1.4
plt.rcParams['legend.handlelength'] = 0.5

In [7]:
# making a new colomap -> tmap

n = 35
diff = 0.5
cmap = plt.cm.BrBG
cmap2 = plt.cm.RdBu_r
lower = cmap(np.linspace(0, diff, n))
lower2 = cmap2(np.linspace(0, diff, n))
white = np.ones((5, 4))
white2 = np.ones((5, 4))
upper = cmap(np.linspace(1-diff, 1, n))
upper2 = cmap2(np.linspace(1-diff, 1, n))
colors = np.vstack((lower, white, upper))
colors2 = np.vstack((lower2, white2, upper2))
tmap = m.colors.LinearSegmentedColormap.from_list('map_white', colors)
tmap2 = m.colors.LinearSegmentedColormap.from_list('map_white', colors2)

In [8]:
def get_events_precip(p95, preciparr):
    idxs = np.where(preciparr >= p95)
    events = preciparr[idxs]
    return events, idxs

# util function to calculate value of qs
def get_qs(temp, pres):
    a1 = 6.1114
    temp0 = 273.16
    a3w = 17.269
    a4w = 35.86
    a3i = 21.875
    a4i = 7.66

    # calculating saturation vapor pressure using temperature values
    if temp > temp0:
        a3 = a3w
        a4 = a4w
        es = a1 * np.exp(a3 * ((temp - temp0)/(temp - a4)))
    elif temp < temp0 - 23:
        a3 = a3i
        a4 = a4i
        es = a1 * np.exp(a3 * ((temp - temp0)/(temp - a4)))
    else:
        esw = a1 * np.exp(a3w * ((temp - temp0)/(temp - a4w)))
        esi = a1 * np.exp(a3i * ((temp - temp0)/(temp - a4i)))
        es = esi + ((esw - esi)*(((temp - (temp0 - 23))/23)**2))

    # get saturation specific humidity value
    epsilon = 0.622
    qs = (epsilon * es) / (pres - ((1 - epsilon)*es))
    return qs

def calc_qs(temp, pres):
    pres_range = len(pres)
    time_range = len(temp)
    qs = np.empty((time_range, pres_range))
    for i in range(time_range):
        for j in range(pres_range):
            qs[i, j] = get_qs(temp[i, j], pres[j])
    return qs

# vert integral function (Simpson's method)
def vert_integ(x, y):
    int = integrate.simpson(y, x, even='avg')

    return int

# finite differnce methods to find derivative
def centered_diff(arr):
    arr_diff = np.empty(len(arr) - 2)
    for i in range((len(arr) - 2)):
        arr_diff[i] = arr[i+2] - arr[i]
    return arr_diff

def forward_diff(arr):
    arr_diff = np.diff(arr)
    return arr_diff

def backward_diff(arr):
    arr_diff = -(np.diff(arr[::-1])[::-1])
    return arr_diff

def get_pe1(omega, pres, qs):
    p_cdiff = centered_diff(pres)
    p_fdiff = forward_diff(pres)
    p_bdiff = backward_diff(pres)

    time_range = len(omega)
    pe = np.empty(time_range)
    thermo = np.empty(time_range)

    # taking mean omega of all extremes to get the thermodynamic contribution
    omega_mean = np.nanmean(omega)

    for i in range(time_range):
        qs_cdiff = centered_diff(qs[i])/(p_cdiff)
        qs_fdiff = forward_diff(qs[i])/(p_fdiff)
        qs_bdiff = backward_diff(qs[i])/(p_bdiff)

        qs_diff = np.insert(qs_cdiff, 0, qs_fdiff[0])
        qs_diff = np.append(qs_diff, qs_bdiff[-1])

        # TODO VARY: the value of 3600 will change for different time calculations
        # 1 hour -> 3600s
        # 3 hour -> 3600*3 and so on
        # '+' sign as pressure is from surface-to-top and not top-to-surface
        pe[i] = (-1/(9.806)) * vert_integ(pres, omega[i]*qs_diff) * 3600
        thermo[i] = (-1/(9.806)) * vert_integ(pres, omega_mean*qs_diff) * 3600

    dyn = pe - thermo
    return pe, dyn, thermo

In [9]:
def get_totals_freq(precip, t2m, d2m, pres, temp_levels, q, vimd, omega):
    print("Starting the scaling process ...")

    print("Initializing zero arrays ...")

    xrange = len(precip[0])
    yrange = len(precip[0][0])

    # initialising the for loop by making zeros array for t2m and d2m to mutate
    precip95 = np.empty((xrange, yrange))
    precip_sum = np.empty((xrange, yrange))
    precip_std = np.empty((xrange, yrange))
    t2m_sum = np.empty((xrange, yrange))
    t2m_std = np.empty((xrange, yrange))
    d2m_sum = np.empty((xrange, yrange))
    d2m_std = np.empty((xrange, yrange))
    vimd_sum = np.empty((xrange, yrange))
    vimd_std = np.empty((xrange, yrange))
    pe_sum = np.empty((xrange, yrange))
    pe_std = np.empty((xrange, yrange))
    dyn_sum = np.empty((xrange, yrange))
    dyn_std = np.empty((xrange, yrange))
    thermo_sum = np.empty((xrange, yrange))
    thermo_std = np.empty((xrange, yrange))
    frequency = np.empty((xrange, yrange))

    print("Starting the loop ...")

    # starting loop
    for lat in tqdm_notebook(xrange(xrange)):
        for lon in tqdm_notebook(xrange(yrange), leave = False):

            # redefine for convenience
            preciparr = precip.isel(lat = lat, lon = lon)
            t2marr = t2m.isel(lat = lat, lon = lon)
            d2marr = d2m.isel(lat = lat, lon = lon)
            temparr = temp_levels.isel(lat = lat, lon = lon)
            qarr = q.isel(lat = lat, lon = lon)
            vimdarr = vimd.isel(lat = lat, lon = lon)
            omegaarr = omega.isel(lat = lat, lon = lon)

            # start
            p95 = preciparr[~np.isnan(preciparr)].quantile(0.95, interpolation='higher')
            precip_events, precip_idxs = get_events_precip(p95, preciparr[~np.isnan(preciparr)])
            no_of_events = len(precip_events)
            precip_events_sum = np.sum(precip_events)
            precip_events_std = np.std(precip_events)
            t2m_events = t2marr[precip_idxs]
            t2m_events_sum = np.sum(t2m_events)
            t2m_events_std = np.std(t2m_events)
            d2m_events = d2marr[precip_idxs]
            d2m_events_sum = np.sum(d2m_events)
            d2m_events_std = np.std(d2m_events)
            temp_events = temparr[precip_idxs]
            q_events = qarr[precip_idxs]
            vimd_events = vimdarr[precip_idxs]
            vimd_events_sum = np.sum(vimd_events)
            vimd_events_std = np.std(vimd_events)
            omega_events = omegaarr[precip_idxs]

            # get the values of qs
            qs_events  = calc_qs(temp_events, pres)

            # get the value of precipitation estimate
            pe_events, dyn_events, thermo_events = get_pe1(omega_events, pres, qs_events)
            pe_events_sum = np.sum(pe_events)
            pe_events_std = np.std(pe_events)
            dyn_events_sum = np.sum(dyn_events)
            dyn_events_std = np.std(dyn_events)
            thermo_events_sum = np.sum(thermo_events)
            thermo_events_std = np.std(thermo_events)

            #### OUTPUT results

            # make 2-D arrays
            precip95[lat, lon] = p95
            precip_sum[lat, lon] = precip_events_sum
            precip_std[lat, lon] = precip_events_std
            frequency[lat, lon] = no_of_events
            t2m_sum[lat, lon] = t2m_events_sum
            t2m_std[lat, lon] = t2m_events_std
            d2m_sum[lat, lon] = d2m_events_sum
            d2m_std[lat, lon] = d2m_events_std
            vimd_sum[lat, lon] = vimd_events_sum
            vimd_std[lat, lon] = vimd_events_std
            pe_sum[lat, lon] = pe_events_sum
            pe_std[lat, lon] = pe_events_std
            dyn_sum[lat, lon] = dyn_events_sum
            dyn_std[lat, lon] = dyn_events_std
            thermo_sum[lat, lon] = thermo_events_sum
            thermo_std[lat, lon] = thermo_events_std

            # print(f"Completed {lat+1}/{xrange} lat and {lon+1}/{yrange} lon", end='\r')
            sleep(0.01)

    # return all the values as  dictionary
    res = {
            # general values and scaling
            "precip_95" : precip95, # 95th percentile precipitation obtaied from new scaling method
            "precip_sum" : precip_sum, # sum of all extreme events at all grid points
            "precip_std" : precip_std, # sum of all extreme events at all grid points
            "frequency" : frequency, # no.of events
            "t2m_sum" : t2m_sum, # sum of all vimd associated with extremes at all grid points
            "t2m_std" : t2m_std, # sum of all vimd associated with extremes at all grid points
            "d2m_sum" : d2m_sum, # sum of all vimd associated with extremes at all grid points
            "d2m_std" : d2m_std, # sum of all vimd associated with extremes at all grid points
            "vimd_sum" : vimd_sum, # sum of all vimd associated with extremes at all grid points
            "vimd_std" : vimd_std, # sum of all vimd associated with extremes at all grid points
            "pe_sum" : pe_sum, # sum of all precipitation estimates obtained using METHOD-1
            "pe_std" : pe_std, # sum of all precipitation estimates obtained using METHOD-1
            "dyn_sum" : dyn_sum, # sum of all dyn estimates obtained using METHOD-1
            "dyn_std" : dyn_std, # sum of all dyn estimates obtained using METHOD-1
            "thermo_sum" : thermo_sum, # sum of all thermo estimates obtained using METHOD-1
            "thermo_std" : thermo_std, # sum of all thermo estimates obtained using METHOD-1
            }

    return res

In [10]:
mfdata_DIR = './data/GPM_lowres_data/gpm_2000.nc'
gpm = xr.open_mfdataset(mfdata_DIR, chunks=dict(time=-1, lat=-1, lon=-1))

In [11]:
precip = gpm.precipCal.resample(time = '24H').sum() / 2
precip = precip.where(precip > 1)
precip = precip.chunk(dict(time = -1, lat = -1, lon=-1))
precip

<xarray.DataArray 'precipCal' (time: 214, lat: 161, lon: 161)>
dask.array<rechunk-merge, shape=(214, 161, 161), dtype=float32, chunksize=(214, 161, 161), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2000-06-01 2000-06-02 ... 2000-12-31
  * lon      (lon) float64 60.0 60.25 60.5 60.75 61.0 ... 99.25 99.5 99.75 100.0
  * lat      (lat) float64 0.0 0.25 0.5 0.75 1.0 ... 39.0 39.25 39.5 39.75 40.0

In [12]:
mfdata_DIR2 = './data/era_data/era_2000.nc'
era = xr.open_dataset(mfdata_DIR2, chunks=dict(time=-1, lat=-1, lon=-1))
# era = era.sel(expver=1, drop=True)
era = era.transpose('time', 'latitude', 'longitude')
# era = era.rename_dims({'longitude':'lon', 'latitude':'lat'})
era = era.rename({'longitude':'lon', 'latitude':'lat'})
era = era.sel(time = slice("2000-06-01 00:00:00","2021-06-30 23:00:00"))
era

<xarray.Dataset>
Dimensions:  (lon: 161, lat: 161, time: 5136)
Coordinates:
  * lon      (lon) float32 60.0 60.25 60.5 60.75 61.0 ... 99.25 99.5 99.75 100.0
  * lat      (lat) float32 40.0 39.75 39.5 39.25 39.0 ... 1.0 0.75 0.5 0.25 0.0
  * time     (time) datetime64[ns] 2000-06-01 ... 2000-12-31T23:00:00
Data variables:
    u10      (time, lat, lon) float32 dask.array<chunksize=(5136, 161, 161), meta=np.ndarray>
    v10      (time, lat, lon) float32 dask.array<chunksize=(5136, 161, 161), meta=np.ndarray>
    d2m      (time, lat, lon) float32 dask.array<chunksize=(5136, 161, 161), meta=np.ndarray>
    t2m      (time, lat, lon) float32 dask.array<chunksize=(5136, 161, 161), meta=np.ndarray>
    mx2t     (time, lat, lon) float32 dask.array<chunksize=(5136, 161, 161), meta=np.ndarray>
    mn2t     (time, lat, lon) float32 dask.array<chunksize=(5136, 161, 161), meta=np.ndarray>
    sst      (time, lat, lon) float32 dask.array<chunksize=(5136, 161, 161), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2021-10-02 12:58:09 GMT by grib_to_netcdf-2.20.0: /opt/ecmw...

In [13]:
mfdata_DIR3 = './data/era_pres_data/era_pres_2000*.nc'
era_pres = xr.open_mfdataset(mfdata_DIR3, chunks=dict(time=-1, lat=-1, lon=-1))
# era = xr.open_mfdataset(mfdata_DIR2, chunks=dict(time=1000, lat=40, lon=40), engine='netcdf4', parallel=True)
# era = xr.open_mfdataset(mfdata_DIR2, engine='netcdf4', combine='nested', concat_dim='time', parallel=True)
# era_pres = xr.open_mfdataset(mfdata_DIR3, chunks=dict(time=1000, lat=40, lon=40))
era_pres = era_pres.transpose('time', 'level', 'latitude', 'longitude')
# era_pres = era_pres.rename_dims({'longitude':'lon', 'latitude':'lat'})
era_pres = era_pres.rename({'longitude':'lon', 'latitude':'lat'})
era_pres = era_pres.sel(time = slice("2000-06-01 00:00:00","2021-06-30 23:00:00"))
era_pres

<xarray.Dataset>
Dimensions:  (lon: 161, lat: 161, level: 29, time: 5136)
Coordinates:
  * lon      (lon) float32 60.0 60.25 60.5 60.75 61.0 ... 99.25 99.5 99.75 100.0
  * lat      (lat) float32 40.0 39.75 39.5 39.25 39.0 ... 1.0 0.75 0.5 0.25 0.0
  * level    (level) int32 50 70 100 125 150 175 ... 875 900 925 950 975 1000
  * time     (time) datetime64[ns] 2000-06-01 ... 2000-12-31T23:00:00
Data variables:
    q        (time, level, lat, lon) float32 dask.array<chunksize=(720, 29, 161, 161), meta=np.ndarray>
    t        (time, level, lat, lon) float32 dask.array<chunksize=(720, 29, 161, 161), meta=np.ndarray>
    w        (time, level, lat, lon) float32 dask.array<chunksize=(720, 29, 161, 161), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2021-11-11 08:18:32 GMT by grib_to_netcdf-2.23.0: /opt/ecmw...

In [15]:
mfdata_DIR4 = './data/era_data2/era_2000.nc'
era2 = xr.open_dataset(mfdata_DIR4, chunks=dict(time=-1, lat=-1, lon=-1))
# era = xr.open_mfdataset(mfdata_DIR2, chunks=dict(time=1000, lat=40, lon=40), engine='netcdf4', parallel=True)
# era2 = xr.open_mfdataset(mfdata_DIR4, engine='netcdf4', combine='nested', concat_dim='time', parallel=True)
# era2 = xr.open_dataset(mfdata_DIR4, chunks=dict(time=-1, lat=-1, lon=-1))
# era2 = era2.sel(expver=1, drop=True)
era2 = era2.transpose('time', 'latitude', 'longitude')
# era2 = era2.rename_dims({'longitude':'lon', 'latitude':'lat'})
era2 = era2.rename({'longitude':'lon', 'latitude':'lat'})
era2 = era2.sel(time = slice("2000-06-01 00:00:00","2021-06-30 23:00:00"))
era2

<xarray.Dataset>
Dimensions:  (lon: 161, lat: 161, time: 5136)
Coordinates:
  * lon      (lon) float32 60.0 60.25 60.5 60.75 61.0 ... 99.25 99.5 99.75 100.0
  * lat      (lat) float32 40.0 39.75 39.5 39.25 39.0 ... 1.0 0.75 0.5 0.25 0.0
  * time     (time) datetime64[ns] 2000-06-01 ... 2000-12-31T23:00:00
Data variables:
    e        (time, lat, lon) float32 dask.array<chunksize=(5136, 161, 161), meta=np.ndarray>
    vimd     (time, lat, lon) float32 dask.array<chunksize=(5136, 161, 161), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-02-04 12:51:49 GMT by grib_to_netcdf-2.23.0: /opt/ecmw...

In [16]:
t2m = era.t2m.resample(time = '24H').mean()
d2m = era.d2m.resample(time = '24H').mean()
vimd = era2.vimd.resample(time = '24H').mean()
pres = era_pres.coords['level'],
temp = era_pres.t.resample(time = '24H').mean()
omega = era_pres.w.resample(time = '24H').mean()
q = era_pres.q.resample(time = '24H').mean()

In [22]:
vimd

<xarray.DataArray 'vimd' (time: 214, lat: 161, lon: 161)>
dask.array<stack, shape=(214, 161, 161), dtype=float32, chunksize=(1, 161, 161), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2000-06-01 2000-06-02 ... 2000-12-31
  * lon      (lon) float32 60.0 60.25 60.5 60.75 61.0 ... 99.25 99.5 99.75 100.0
  * lat      (lat) float32 40.0 39.75 39.5 39.25 39.0 ... 1.0 0.75 0.5 0.25 0.0

In [17]:
ds_comb = xr.merge([precip, t2m, d2m, vimd])
ds_comb2 = xr.merge([temp, q, omega])
ds_comb2

,Array,Chunk
Bytes,613.65 MiB,2.87 MiB
Shape,"(214, 29, 161, 161)","(1, 29, 161, 161)"
Count,924 Tasks,214 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,613.65 MiB,2.87 MiB
Shape,"(214, 29, 161, 161)","(1, 29, 161, 161)"
Count,924 Tasks,214 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [18]:
mon = [6,7,8,9]
premon = [3,4,5]
postmon = [10,11,12]
wint = [1,2]

In [23]:
ds_comb_mon = xr.concat([list(ds_comb.groupby('time.month'))[i-1][1] for i in mon], dim='time')
ds_comb_mon2 = xr.concat([list(ds_comb2.groupby('time.month'))[i-1][1] for i in mon], dim='time')

IndexError: list index out of range

In [ ]:
ds_comb_mon = ds_comb_mon.chunk(dict(time = -1)).sortby('time.year')
ds_comb_mon2 = ds_comb_mon2.chunk(dict(time = -1)).sortby('time.year')
ds_comb_mon2

In [ ]:
ds_comb_mon_loaded = ds_comb_mon.load()
ds_comb_mon2_loaded = ds_comb_mon2.load()

In [ ]:
precip = ds_comb_mon_loaded.precipCal
t2m = ds_comb_mon_loaded.t2m
d2m = ds_comb_mon_loaded.d2m
vimc = -ds_comb_mon_loaded.vimd

temp = ds_comb_mon2_loaded.t
q = ds_comb_mon2_loaded.q
omega = ds_comb_mon2_loaded.w

In [ ]:
q.sel(level = 850, time = '2013-06-17 00:00:00').plot.contourf(cmap = 'Greens', levels=9)

In [ ]:
pres = ds_comb_mon2_loaded.coords['level'].to_numpy()
pres

In [35]:
res = get_mean_freq(precip, t2m, d2m, pres, temp, q, vimd, omega)

Starting the scaling process ...
Initializing zero arrays ...
Starting the loop ...


KeyboardInterrupt: 